# 2. Feature Engineering

Ce notebook gère la transformation des données textuelles en features numériques pour la classification

## Import des bibliothèques

In [1]:
import pandas as pd
import numpy as np
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
import warnings
warnings.filterwarnings('ignore')
print("Bibliothèques importées avec succès")

Bibliothèques importées avec succès


## Chargement des données preprocessed

In [2]:
# Charger les données preprocessed
with open('pkl/preprocessed_data.pkl', 'rb') as f:
    preprocessed_data = pickle.load(f)

X_train = preprocessed_data['X_train']
X_test = preprocessed_data['X_test']
y_train = preprocessed_data['y_train']
y_test = preprocessed_data['y_test']
label_encoder = preprocessed_data['label_encoder']

print(f"Données chargées avec succès")
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_test shape: {y_test.shape}")

Données chargées avec succès
X_train shape: (1966,)
X_test shape: (492,)
y_train shape: (1966,)
y_test shape: (492,)


## Feature Engineering 1: TF-IDF Vectorization

In [3]:
# Créer le vectoriseur TF-IDF
print("Création des features TF-IDF...")

tfidf_vectorizer = TfidfVectorizer(
    max_features=5000,  # Limiter à 5000 features les plus importantes
    min_df=2,  # Ignorer les termes qui apparaissent dans moins de 2 documents
    max_df=0.8,  # Ignorer les termes qui apparaissent dans plus de 80% des documents
    ngram_range=(1, 2),  # Utiliser des unigrammes et bigrammes
    sublinear_tf=True  # Utiliser l'échelle logarithmique pour les fréquences
)

# Fit sur l'ensemble d'entraînement et transform les deux ensembles
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

print(f"\nShape des features TF-IDF:")
print(f"X_train_tfidf: {X_train_tfidf.shape}")
print(f"X_test_tfidf: {X_test_tfidf.shape}")

Création des features TF-IDF...

Shape des features TF-IDF:
X_train_tfidf: (1966, 5000)
X_test_tfidf: (492, 5000)


In [4]:
# Afficher les top features TF-IDF
feature_names = tfidf_vectorizer.get_feature_names_out()
print(f"\nNombre total de features TF-IDF: {len(feature_names)}")
print(f"\nTop 20 features:")
print(feature_names[:20])


Nombre total de features TF-IDF: 5000

Top 20 features:
['27001' '27001 foundation' '3d' '3d contribue' '3d unreal' '9001' 'abr'
 'academic' 'academic coordinator' 'académiques' 'acca' 'access'
 'access control' 'accessibility' 'accessibility git' 'accessibilité'
 'accessibilité des' 'accompagne' 'accompagne la' 'accompagne les']


## Feature Engineering 2: Count Vectorization

In [5]:
# Créer le vectoriseur Count
print("Création des features Count Vectorizer...")

count_vectorizer = CountVectorizer(
    max_features=3000,
    min_df=2,
    max_df=0.8,
    ngram_range=(1, 2)
)

# Fit sur l'ensemble d'entraînement et transform les deux ensembles
X_train_count = count_vectorizer.fit_transform(X_train)
X_test_count = count_vectorizer.transform(X_test)

print(f"\nShape des features Count:")
print(f"X_train_count: {X_train_count.shape}")
print(f"X_test_count: {X_test_count.shape}")

Création des features Count Vectorizer...

Shape des features Count:
X_train_count: (1966, 3000)
X_test_count: (492, 3000)


## Feature Engineering 3: TF-IDF avec réduction de dimensionnalité (SVD)

In [6]:
# Créer un TF-IDF avec plus de features pour la réduction
print("Création des features TF-IDF + SVD...")

tfidf_vectorizer_svd = TfidfVectorizer(
    max_features=10000,
    min_df=2,
    max_df=0.8,
    ngram_range=(1, 3),
    sublinear_tf=True
)

X_train_tfidf_full = tfidf_vectorizer_svd.fit_transform(X_train)
X_test_tfidf_full = tfidf_vectorizer_svd.transform(X_test)

print(f"Shape avant SVD:")
print(f"X_train_tfidf_full: {X_train_tfidf_full.shape}")

Création des features TF-IDF + SVD...
Shape avant SVD:
X_train_tfidf_full: (1966, 10000)


In [7]:
# Appliquer TruncatedSVD pour réduire la dimensionnalité
svd = TruncatedSVD(n_components=300, random_state=42)

X_train_svd = svd.fit_transform(X_train_tfidf_full)
X_test_svd = svd.transform(X_test_tfidf_full)

print(f"\nShape après SVD:")
print(f"X_train_svd: {X_train_svd.shape}")
print(f"X_test_svd: {X_test_svd.shape}")
print(f"\nVariance expliquée par les composantes: {svd.explained_variance_ratio_.sum():.4f}")


Shape après SVD:
X_train_svd: (1966, 300)
X_test_svd: (492, 300)

Variance expliquée par les composantes: 0.5035


## Feature Engineering 4: Features statistiques

In [8]:
# Créer des features statistiques basiques
def create_statistical_features(text_series):
    """
    Crée des features statistiques à partir du texte
    """
    features = pd.DataFrame()
    
    # Longueur du texte
    features['text_length'] = text_series.apply(len)
    
    # Nombre de mots
    features['word_count'] = text_series.apply(lambda x: len(x.split()))
    
    # Longueur moyenne des mots
    features['avg_word_length'] = text_series.apply(
        lambda x: np.mean([len(word) for word in x.split()]) if len(x.split()) > 0 else 0
    )
    
    # Nombre de mots uniques
    features['unique_words'] = text_series.apply(
        lambda x: len(set(x.split()))
    )
    
    # Ratio de mots uniques
    features['unique_ratio'] = features['unique_words'] / (features['word_count'] + 1)
    
    return features.values

print("Création des features statistiques...")

X_train_stats = create_statistical_features(X_train)
X_test_stats = create_statistical_features(X_test)

print(f"\nShape des features statistiques:")
print(f"X_train_stats: {X_train_stats.shape}")
print(f"X_test_stats: {X_test_stats.shape}")

Création des features statistiques...

Shape des features statistiques:
X_train_stats: (1966, 5)
X_test_stats: (492, 5)


## Combinaison de features: TF-IDF + Features statistiques

In [9]:
# Combiner TF-IDF avec features statistiques
from scipy.sparse import hstack, csr_matrix

print("Combinaison TF-IDF + Features statistiques...")

X_train_combined = hstack([X_train_tfidf, csr_matrix(X_train_stats)])
X_test_combined = hstack([X_test_tfidf, csr_matrix(X_test_stats)])

print(f"\nShape des features combinées:")
print(f"X_train_combined: {X_train_combined.shape}")
print(f"X_test_combined: {X_test_combined.shape}")

Combinaison TF-IDF + Features statistiques...

Shape des features combinées:
X_train_combined: (1966, 5005)
X_test_combined: (492, 5005)


## Sauvegarde de toutes les représentations de features

In [10]:
# Sauvegarder tous les ensembles de features et les vectorizers
feature_sets = {
    # Ensembles de features
    'X_train_tfidf': X_train_tfidf,
    'X_test_tfidf': X_test_tfidf,
    'X_train_count': X_train_count,
    'X_test_count': X_test_count,
    'X_train_svd': X_train_svd,
    'X_test_svd': X_test_svd,
    'X_train_stats': X_train_stats,
    'X_test_stats': X_test_stats,
    'X_train_combined': X_train_combined,
    'X_test_combined': X_test_combined,
    
    # Labels
    'y_train': y_train,
    'y_test': y_test,
    
    # Vectorizers et transformers
    'tfidf_vectorizer': tfidf_vectorizer,
    'count_vectorizer': count_vectorizer,
    'tfidf_vectorizer_svd': tfidf_vectorizer_svd,
    'svd': svd,
    'label_encoder': label_encoder
}

with open('pkl/feature_sets.pkl', 'wb') as f:
    pickle.dump(feature_sets, f)

print("Tous les ensembles de features sauvegardés: pkl/feature_sets.pkl")

Tous les ensembles de features sauvegardés: pkl/feature_sets.pkl


## Résumé du Feature Engineering

In [11]:
print("="*60)
print("RÉSUMÉ DU FEATURE ENGINEERING")
print("="*60)

print("\n1. REPRÉSENTATIONS DE FEATURES CRÉÉES:")
print("   a) TF-IDF (5000 features):")
print(f"      - Train: {X_train_tfidf.shape}")
print(f"      - Test: {X_test_tfidf.shape}")

print("\n   b) Count Vectorizer (3000 features):")
print(f"      - Train: {X_train_count.shape}")
print(f"      - Test: {X_test_count.shape}")

print("\n   c) TF-IDF + SVD (300 composantes):")
print(f"      - Train: {X_train_svd.shape}")
print(f"      - Test: {X_test_svd.shape}")
print(f"      - Variance expliquée: {svd.explained_variance_ratio_.sum():.4f}")

print("\n   d) Features statistiques (5 features):")
print(f"      - Train: {X_train_stats.shape}")
print(f"      - Test: {X_test_stats.shape}")

print("\n   e) TF-IDF + Stats combinées (5005 features):")
print(f"      - Train: {X_train_combined.shape}")
print(f"      - Test: {X_test_combined.shape}")

print("\n2. FICHIERS SAUVEGARDÉS:")
print("   - pkl/feature_sets.pkl (tous les ensembles de features)")

print("\n3. PRÊT POUR LE MODELING:")
print("   - 5 représentations différentes de features")
print("   - Tous les vectorizers/transformers sauvegardés")
print("   - Labels d'entraînement et de test disponibles")

print("\n" + "="*60)
print("FEATURE ENGINEERING TERMINÉ AVEC SUCCÈS")
print("="*60)

RÉSUMÉ DU FEATURE ENGINEERING

1. REPRÉSENTATIONS DE FEATURES CRÉÉES:
   a) TF-IDF (5000 features):
      - Train: (1966, 5000)
      - Test: (492, 5000)

   b) Count Vectorizer (3000 features):
      - Train: (1966, 3000)
      - Test: (492, 3000)

   c) TF-IDF + SVD (300 composantes):
      - Train: (1966, 300)
      - Test: (492, 300)
      - Variance expliquée: 0.5035

   d) Features statistiques (5 features):
      - Train: (1966, 5)
      - Test: (492, 5)

   e) TF-IDF + Stats combinées (5005 features):
      - Train: (1966, 5005)
      - Test: (492, 5005)

2. FICHIERS SAUVEGARDÉS:
   - pkl/feature_sets.pkl (tous les ensembles de features)

3. PRÊT POUR LE MODELING:
   - 5 représentations différentes de features
   - Tous les vectorizers/transformers sauvegardés
   - Labels d'entraînement et de test disponibles

FEATURE ENGINEERING TERMINÉ AVEC SUCCÈS
